# 🔄 Building a Complete Data Pipeline

Welcome to the **sixth tutorial** in our Data Ingestion Pipeline series! In this comprehensive notebook, you'll learn how to orchestrate all the components we've built into a complete, production-ready data pipeline.

## 🎯 Learning Objectives

By the end of this tutorial, you will:
- ✅ Understand pipeline orchestration and workflow management
- ✅ Build a complete end-to-end data pipeline
- ✅ Implement error handling and recovery mechanisms
- ✅ Add performance monitoring and optimization
- ✅ Create comprehensive pipeline reporting
- ✅ Test and validate your complete pipeline

---

## 🏗️ Pipeline Architecture Overview

Before we start building, let's understand the complete architecture of our data pipeline:

```
📥 DATA SOURCES          🔄 PIPELINE STAGES           📊 OUTPUTS
┌─────────────────┐     ┌──────────────────────┐     ┌─────────────────┐
│ 📁 CSV Files    │────▶│ 1️⃣ Data Ingestion    │────▶│ 🗄️ Database     │
│ 📄 JSON Files   │     │ 2️⃣ Data Validation   │     │ 📊 Reports      │
│ 🌐 REST APIs    │     │ 3️⃣ Data Transformation│     │ 📈 Analytics    │
│ 🗄️ Databases    │     │ 4️⃣ Data Storage      │     │ 🚨 Alerts       │
└─────────────────┘     │ 5️⃣ Monitoring        │     └─────────────────┘
                        └──────────────────────┘
```

### 🎯 **Pipeline Goals**
- **Reliability**: Handle failures gracefully
- **Scalability**: Process large volumes of data
- **Maintainability**: Easy to understand and modify
- **Observability**: Monitor performance and health
- **Flexibility**: Support multiple data sources and formats

In [ ]:
# Import all necessary libraries
import sys
import os
import pandas as pd
import numpy as np
import json
import logging
import time
from datetime import datetime, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any, Optional

# Add src to path for imports
sys.path.append(os.path.join('..', 'src'))

# Import our pipeline components
try:
    from pipeline.pipeline_manager import PipelineManager
    from ingestion.file_ingestion import FileIngestion
    from ingestion.api_ingestion import APIIngestion
    from validation.data_validator import DataValidator
    from transformation.data_cleaner import DataCleaner
    from transformation.data_enricher import DataEnricher
    from storage.database_manager import DatabaseManager
    from utils.config import config
    from utils.helpers import ensure_directory_exists, format_duration
    print("✅ All pipeline components imported successfully!")
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    print("📝 Note: Some components may not be available in this demo environment")

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

print("🚀 Pipeline building environment ready!")

## 🔧 Step 1: Pipeline Configuration

First, let's set up our pipeline configuration. This includes defining data sources, processing parameters, and output destinations.

In [ ]:
# Pipeline Configuration
class PipelineConfig:
    """Configuration class for our data pipeline"""
    
    def __init__(self):
        # Data source configuration
        self.data_sources = {
            'csv_files': {
                'enabled': True,
                'path': '../data/input/csv/',
                'pattern': '*.csv',
                'encoding': 'utf-8'
            },
            'json_files': {
                'enabled': True,
                'path': '../data/input/json/',
                'pattern': '*.json'
            },
            'api_sources': {
                'enabled': True,
                'endpoints': [
                    'https://jsonplaceholder.typicode.com/posts'
                ],
                'timeout': 30,
                'retry_attempts': 3
            }
        }
        
        # Processing configuration
        self.processing = {
            'batch_size': 1000,
            'max_workers': 4,
            'enable_parallel': True,
            'validation_threshold': 0.8,  # 80% quality threshold
            'enable_enrichment': True,
            'enable_standardization': True
        }
        
        # Output configuration
        self.outputs = {
            'database': {
                'enabled': True,
                'path': '../data/orders.db',
                'table': 'orders'
            },
            'files': {
                'enabled': True,
                'output_dir': '../data/output/',
                'formats': ['csv', 'json']
            },
            'reports': {
                'enabled': True,
                'report_dir': '../data/output/reports/'
            }
        }
        
        # Monitoring configuration
        self.monitoring = {
            'log_level': 'INFO',
            'log_file': '../logs/pipeline.log',
            'metrics_enabled': True,
            'alerts_enabled': False,  # Disable for demo
            'performance_tracking': True
        }

# Initialize configuration
pipeline_config = PipelineConfig()

print("⚙️ Pipeline Configuration:")
print("=" * 50)
print(f"📁 Data Sources: {len(pipeline_config.data_sources)} configured")
print(f"🔄 Batch Size: {pipeline_config.processing['batch_size']}")
print(f"👥 Max Workers: {pipeline_config.processing['max_workers']}")
print(f"📊 Quality Threshold: {pipeline_config.processing['validation_threshold']*100}%")
print(f"💾 Database Output: {'✅ Enabled' if pipeline_config.outputs['database']['enabled'] else '❌ Disabled'}")
print(f"📈 Monitoring: {'✅ Enabled' if pipeline_config.monitoring['metrics_enabled'] else '❌ Disabled'}")

## 📊 Step 2: Create Sample Data

Let's create some sample data to demonstrate our pipeline. In a real scenario, this data would come from your actual sources.

In [ ]:
# Create sample data for demonstration
import pandas as pd
import json
import os
from pathlib import Path

def ensure_directory_exists(directory_path):
    """Ensure directory exists, create if it doesn't"""
    Path(directory_path).mkdir(parents=True, exist_ok=True)

def create_sample_data():
    """Create sample data files for pipeline demonstration"""

    # Get the current working directory and construct paths
    current_dir = Path.cwd()

    # If we're in notebooks folder, go up one level
    if current_dir.name == 'notebooks':
        base_dir = current_dir.parent
    else:
        base_dir = current_dir

    # Define data directories
    csv_dir = base_dir / 'data' / 'input' / 'csv'
    json_dir = base_dir / 'data' / 'input' / 'json'
    output_dir = base_dir / 'data' / 'output'
    logs_dir = base_dir / 'logs'

    # Ensure directories exist
    ensure_directory_exists(csv_dir)
    ensure_directory_exists(json_dir)
    ensure_directory_exists(output_dir)
    ensure_directory_exists(logs_dir)

    # Sample CSV data (Store orders)
    csv_data = {
        'order_id': ['ORD-2024-001', 'ORD-2024-002', 'ORD-2024-003', 'ORD-2024-004', 'ORD-2024-005'],
        'customer_name': ['John Doe', 'Jane Smith', 'Bob Wilson', 'Alice Johnson', 'Charlie Brown'],
        'product': ['iPhone 15', 'MacBook Pro', 'AirPods Pro', 'iPad Air', 'Apple Watch'],
        'quantity': [1, 1, 2, 1, 1],
        'price': [999.99, 1999.99, 249.99, 599.99, 399.99],
        'order_date': ['2024-01-15', '2024-01-15', '2024-01-16', '2024-01-16', '2024-01-17'],
        'store_location': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'],
        'customer_email': ['john@example.com', 'jane@example.com', 'bob@example.com', 'alice@example.com', 'charlie@example.com']
    }

    df_csv = pd.DataFrame(csv_data)
    csv_file_path = csv_dir / 'store_orders_2024.csv'
    df_csv.to_csv(csv_file_path, index=False)

    # Sample JSON data (Mobile app orders)
    json_data = {
        'app_version': '2.1.0',
        'upload_time': '2024-01-17T12:00:00Z',
        'orders': [
            {
                'order_id': 'APP-2024-001',
                'customer_name': 'Sarah Connor',
                'product': 'Nintendo Switch',
                'quantity': 1,
                'price': 299.99,
                'order_date': '2024-01-17T10:30:00Z',
                'device_type': 'iOS',
                'customer_email': 'sarah@example.com'
            },
            {
                'order_id': 'APP-2024-002',
                'customer_name': 'Mike Johnson',
                'product': 'PlayStation 5',
                'quantity': 1,
                'price': 499.99,
                'order_date': '2024-01-17T11:15:00Z',
                'device_type': 'Android',
                'customer_email': 'mike@example.com'
            },
            {
                'order_id': 'APP-2024-003',
                'customer_name': 'Lisa Davis',
                'product': 'Xbox Series X',
                'quantity': 1,
                'price': 499.99,
                'order_date': '2024-01-17T14:20:00Z',
                'device_type': 'iOS',
                'customer_email': 'lisa@example.com'
            }
        ]
    }

    json_file_path = json_dir / 'mobile_orders_2024.json'
    with open(json_file_path, 'w') as f:
        json.dump(json_data, f, indent=2)

    print(f"✅ Created CSV file: {csv_file_path}")
    print(f"✅ Created JSON file: {json_file_path}")

    return len(csv_data['order_id']), len(json_data['orders'])

# Create sample data
try:
    csv_count, json_count = create_sample_data()

    print("\n📊 Sample Data Created:")
    print("=" * 30)
    print(f"📁 CSV Orders: {csv_count}")
    print(f"📄 JSON Orders: {json_count}")
    print(f"📈 Total Orders: {csv_count + json_count}")
    print("\n✅ Sample data files created successfully!")

except Exception as e:
    print(f"❌ Error creating sample data: {e}")
    print(f"Current working directory: {Path.cwd()}")
    print("Please make sure you're running this from the project root or notebooks directory")

## 🔄 Step 3: Build Pipeline Components

Now let's build our pipeline step by step, integrating all the components we've learned about in previous tutorials.

In [ ]:
# Custom Pipeline Builder Class
class DataPipelineBuilder:
    """Custom pipeline builder for demonstration purposes"""
    
    def __init__(self, config):
        self.config = config
        self.pipeline_id = f"PIPELINE-{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.start_time = None
        self.end_time = None
        self.results = {
            'ingestion': {},
            'validation': {},
            'transformation': {},
            'storage': {},
            'monitoring': {}
        }
        
        # Setup logging
        self.setup_logging()
        
    def setup_logging(self):
        """Setup pipeline logging"""
        logging.basicConfig(
            level=getattr(logging, self.config.monitoring['log_level']),
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.StreamHandler(),
                logging.FileHandler(self.config.monitoring['log_file'])
            ]
        )
        self.logger = logging.getLogger(f'Pipeline-{self.pipeline_id}')
        
    def run_ingestion_stage(self):
        """Run data ingestion stage"""
        self.logger.info("🔄 Starting data ingestion stage")
        stage_start = time.time()
        
        all_data = []
        ingestion_summary = {
            'sources_processed': 0,
            'total_records': 0,
            'successful_sources': 0,
            'failed_sources': 0,
            'errors': []
        }
        
        # Process CSV files
        if self.config.data_sources['csv_files']['enabled']:
            try:
                csv_path = self.config.data_sources['csv_files']['path']
                csv_files = list(Path(csv_path).glob('*.csv'))
                
                for csv_file in csv_files:
                    try:
                        df = pd.read_csv(csv_file)
                        df['source'] = 'csv_file'
                        df['source_file'] = csv_file.name
                        df['ingested_at'] = datetime.now().isoformat()
                        all_data.append(df)
                        
                        ingestion_summary['total_records'] += len(df)
                        ingestion_summary['successful_sources'] += 1
                        self.logger.info(f"✅ Processed CSV file: {csv_file.name} ({len(df)} records)")
                        
                    except Exception as e:
                        error_msg = f"Failed to process CSV file {csv_file.name}: {str(e)}"
                        ingestion_summary['errors'].append(error_msg)
                        ingestion_summary['failed_sources'] += 1
                        self.logger.error(f"❌ {error_msg}")
                        
                ingestion_summary['sources_processed'] += len(csv_files)
                
            except Exception as e:
                error_msg = f"CSV ingestion failed: {str(e)}"
                ingestion_summary['errors'].append(error_msg)
                self.logger.error(f"❌ {error_msg}")
        
        # Process JSON files
        if self.config.data_sources['json_files']['enabled']:
            try:
                json_path = self.config.data_sources['json_files']['path']
                json_files = list(Path(json_path).glob('*.json'))
                
                for json_file in json_files:
                    try:
                        with open(json_file, 'r') as f:
                            json_data = json.load(f)
                        
                        # Extract orders from JSON structure
                        if 'orders' in json_data:
                            df = pd.DataFrame(json_data['orders'])
                        else:
                            df = pd.DataFrame([json_data])
                        
                        df['source'] = 'json_file'
                        df['source_file'] = json_file.name
                        df['ingested_at'] = datetime.now().isoformat()
                        
                        # Add JSON metadata if available
                        if 'app_version' in json_data:
                            df['app_version'] = json_data['app_version']
                        
                        all_data.append(df)
                        
                        ingestion_summary['total_records'] += len(df)
                        ingestion_summary['successful_sources'] += 1
                        self.logger.info(f"✅ Processed JSON file: {json_file.name} ({len(df)} records)")
                        
                    except Exception as e:
                        error_msg = f"Failed to process JSON file {json_file.name}: {str(e)}"
                        ingestion_summary['errors'].append(error_msg)
                        ingestion_summary['failed_sources'] += 1
                        self.logger.error(f"❌ {error_msg}")
                        
                ingestion_summary['sources_processed'] += len(json_files)
                
            except Exception as e:
                error_msg = f"JSON ingestion failed: {str(e)}"
                ingestion_summary['errors'].append(error_msg)
                self.logger.error(f"❌ {error_msg}")
        
        # Combine all data
        if all_data:
            combined_data = pd.concat(all_data, ignore_index=True)
            # Remove duplicates based on order_id if present
            if 'order_id' in combined_data.columns:
                before_dedup = len(combined_data)
                combined_data = combined_data.drop_duplicates(subset=['order_id'], keep='first')
                after_dedup = len(combined_data)
                if before_dedup != after_dedup:
                    self.logger.info(f"🧹 Removed {before_dedup - after_dedup} duplicate records")
        else:
            combined_data = pd.DataFrame()
        
        stage_time = time.time() - stage_start
        ingestion_summary['processing_time'] = stage_time
        ingestion_summary['success'] = len(combined_data) > 0
        
        self.results['ingestion'] = ingestion_summary
        
        self.logger.info(f"✅ Ingestion stage completed in {stage_time:.2f}s: {len(combined_data)} records")
        return combined_data
    
    def run_validation_stage(self, data):
        """Run data validation stage"""
        self.logger.info("🔍 Starting data validation stage")
        stage_start = time.time()
        
        validation_summary = {
            'total_records': len(data),
            'valid_records': 0,
            'invalid_records': 0,
            'quality_score': 0.0,
            'errors': [],
            'warnings': []
        }
        
        if data.empty:
            validation_summary['errors'].append("No data to validate")
            validation_summary['success'] = False
            self.results['validation'] = validation_summary
            return data
        
        # Required fields validation
        required_fields = ['order_id', 'customer_name', 'product', 'quantity', 'price']
        missing_fields = [field for field in required_fields if field not in data.columns]
        
        if missing_fields:
            validation_summary['errors'].append(f"Missing required fields: {missing_fields}")
        
        # Data quality checks
        valid_record_count = 0
        
        for index, row in data.iterrows():
            record_valid = True
            
            # Check required fields are not empty
            for field in required_fields:
                if field in row and (pd.isna(row[field]) or str(row[field]).strip() == ''):
                    validation_summary['errors'].append(f"Row {index}: Missing {field}")
                    record_valid = False
            
            # Business rule validations
            if 'quantity' in row and pd.notna(row['quantity']):
                try:
                    quantity = float(row['quantity'])
                    if quantity <= 0:
                        validation_summary['errors'].append(f"Row {index}: Invalid quantity {quantity}")
                        record_valid = False
                except (ValueError, TypeError):
                    validation_summary['errors'].append(f"Row {index}: Non-numeric quantity")
                    record_valid = False
            
            if 'price' in row and pd.notna(row['price']):
                try:
                    price = float(row['price'])
                    if price <= 0:
                        validation_summary['errors'].append(f"Row {index}: Invalid price {price}")
                        record_valid = False
                except (ValueError, TypeError):
                    validation_summary['errors'].append(f"Row {index}: Non-numeric price")
                    record_valid = False
            
            if record_valid:
                valid_record_count += 1
        
        validation_summary['valid_records'] = valid_record_count
        validation_summary['invalid_records'] = len(data) - valid_record_count
        validation_summary['quality_score'] = (valid_record_count / len(data)) * 100 if len(data) > 0 else 0
        
        # Check if quality meets threshold
        quality_threshold = self.config.processing['validation_threshold'] * 100
        validation_summary['meets_threshold'] = validation_summary['quality_score'] >= quality_threshold
        validation_summary['success'] = validation_summary['meets_threshold']
        
        stage_time = time.time() - stage_start
        validation_summary['processing_time'] = stage_time
        
        self.results['validation'] = validation_summary
        
        self.logger.info(f"✅ Validation stage completed in {stage_time:.2f}s: {validation_summary['quality_score']:.1f}% quality")
        
        if not validation_summary['meets_threshold']:
            self.logger.warning(f"⚠️ Data quality ({validation_summary['quality_score']:.1f}%) below threshold ({quality_threshold}%)")
        
        return data

# Initialize pipeline builder
pipeline_builder = DataPipelineBuilder(pipeline_config)
print(f"🔧 Pipeline Builder initialized: {pipeline_builder.pipeline_id}")

## 🧹 Step 4: Data Transformation Stage

Let's add the transformation stage to clean, standardize, and enrich our data.

In [ ]:
# Add transformation methods to our pipeline builder
def run_transformation_stage(self, data):
    """Run data transformation stage"""
    self.logger.info("🧹 Starting data transformation stage")
    stage_start = time.time()
    
    transformation_summary = {
        'original_records': len(data),
        'transformed_records': 0,
        'operations_performed': [],
        'errors': []
    }
    
    if data.empty:
        transformation_summary['errors'].append("No data to transform")
        transformation_summary['success'] = False
        self.results['transformation'] = transformation_summary
        return data
    
    transformed_data = data.copy()
    
    try:
        # 1. Data Cleaning
        # Standardize customer names
        if 'customer_name' in transformed_data.columns:
            transformed_data['customer_name'] = transformed_data['customer_name'].str.strip().str.title()
            transformation_summary['operations_performed'].append('Standardized customer names')
        
        # Clean product names
        if 'product' in transformed_data.columns:
            product_mapping = {
                'iphone 15': 'iPhone 15',
                'macbook pro': 'MacBook Pro',
                'airpods pro': 'AirPods Pro',
                'ipad air': 'iPad Air',
                'apple watch': 'Apple Watch',
                'nintendo switch': 'Nintendo Switch',
                'playstation 5': 'PlayStation 5',
                'xbox series x': 'Xbox Series X'
            }
            
            transformed_data['product'] = transformed_data['product'].str.lower()
            for old_name, new_name in product_mapping.items():
                transformed_data['product'] = transformed_data['product'].replace(old_name, new_name)
            transformation_summary['operations_performed'].append('Standardized product names')
        
        # 2. Data Type Conversion
        if 'quantity' in transformed_data.columns:
            transformed_data['quantity'] = pd.to_numeric(transformed_data['quantity'], errors='coerce')
            transformed_data['quantity'] = transformed_data['quantity'].fillna(1).astype(int)
            transformation_summary['operations_performed'].append('Fixed quantity data types')
        
        if 'price' in transformed_data.columns:
            transformed_data['price'] = pd.to_numeric(transformed_data['price'], errors='coerce')
            transformed_data['price'] = transformed_data['price'].round(2)
            transformation_summary['operations_performed'].append('Fixed price data types')
        
        # 3. Data Enrichment
        if self.config.processing['enable_enrichment']:
            # Calculate total amount
            if 'quantity' in transformed_data.columns and 'price' in transformed_data.columns:
                transformed_data['total_amount'] = transformed_data['quantity'] * transformed_data['price']
                transformation_summary['operations_performed'].append('Added total_amount calculation')
            
            # Add order size category
            if 'total_amount' in transformed_data.columns:
                transformed_data['order_size'] = pd.cut(
                    transformed_data['total_amount'],
                    bins=[0, 100, 500, 1000, 2000, float('inf')],
                    labels=['XSmall', 'Small', 'Medium', 'Large', 'XLarge']
                )
                transformation_summary['operations_performed'].append('Added order size categorization')
            
            # Add customer segment
            if 'total_amount' in transformed_data.columns:
                def categorize_customer(amount):
                    if amount >= 1500:
                        return 'VIP'
                    elif amount >= 800:
                        return 'Premium'
                    elif amount >= 300:
                        return 'Standard'
                    else:
                        return 'Budget'
                
                transformed_data['customer_segment'] = transformed_data['total_amount'].apply(categorize_customer)
                transformation_summary['operations_performed'].append('Added customer segmentation')
            
            # Add processing metadata
            transformed_data['processed_at'] = datetime.now().isoformat()
            transformed_data['pipeline_id'] = self.pipeline_id
            transformation_summary['operations_performed'].append('Added processing metadata')
        
        # 4. Data Quality Score
        def calculate_record_quality(row):
            score = 100
            
            # Deduct for missing optional fields
            optional_fields = ['customer_email', 'store_location']
            for field in optional_fields:
                if field in row and (pd.isna(row[field]) or str(row[field]).strip() == ''):
                    score -= 10
            
            # Deduct for suspicious values
            if 'customer_name' in row and len(str(row['customer_name'])) < 3:
                score -= 15
            
            if 'total_amount' in row and row['total_amount'] > 5000:
                score -= 5  # High value orders might need review
            
            return max(0, score)
        
        transformed_data['quality_score'] = transformed_data.apply(calculate_record_quality, axis=1)
        transformation_summary['operations_performed'].append('Added quality scoring')
        
        # Remove records with critical issues (if any)
        before_filter = len(transformed_data)
        transformed_data = transformed_data.dropna(subset=['order_id', 'customer_name', 'product'])
        after_filter = len(transformed_data)
        
        if before_filter != after_filter:
            removed_count = before_filter - after_filter
            transformation_summary['operations_performed'].append(f'Removed {removed_count} records with critical issues')
            self.logger.warning(f"⚠️ Removed {removed_count} records with critical missing data")
        
        transformation_summary['transformed_records'] = len(transformed_data)
        transformation_summary['success'] = True
        
    except Exception as e:
        error_msg = f"Transformation failed: {str(e)}"
        transformation_summary['errors'].append(error_msg)
        transformation_summary['success'] = False
        self.logger.error(f"❌ {error_msg}")
        transformed_data = data  # Return original data if transformation fails
    
    stage_time = time.time() - stage_start
    transformation_summary['processing_time'] = stage_time
    
    self.results['transformation'] = transformation_summary
    
    self.logger.info(f"✅ Transformation stage completed in {stage_time:.2f}s: {len(transformation_summary['operations_performed'])} operations")
    
    return transformed_data

# Add method to pipeline builder class
DataPipelineBuilder.run_transformation_stage = run_transformation_stage

print("🧹 Transformation stage methods added to pipeline builder")

## 💾 Step 5: Data Storage Stage

Now let's add the storage stage to save our processed data to various outputs.

In [ ]:
# Add storage methods to our pipeline builder
def run_storage_stage(self, data):
    """Run data storage stage"""
    self.logger.info("💾 Starting data storage stage")
    stage_start = time.time()
    
    storage_summary = {
        'total_records': len(data),
        'stored_records': 0,
        'storage_operations': [],
        'errors': []
    }
    
    if data.empty:
        storage_summary['errors'].append("No data to store")
        storage_summary['success'] = False
        self.results['storage'] = storage_summary
        return
    
    try:
        # 1. File Storage
        if self.config.outputs['files']['enabled']:
            output_dir = Path(self.config.outputs['files']['output_dir'])
            output_dir.mkdir(parents=True, exist_ok=True)
            
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            
            # Save as CSV
            if 'csv' in self.config.outputs['files']['formats']:
                csv_file = output_dir / f'processed_orders_{timestamp}.csv'
                data.to_csv(csv_file, index=False)
                storage_summary['storage_operations'].append(f'Saved CSV: {csv_file.name}')
                self.logger.info(f"💾 Saved CSV file: {csv_file}")
            
            # Save as JSON
            if 'json' in self.config.outputs['files']['formats']:
                json_file = output_dir / f'processed_orders_{timestamp}.json'
                # Convert DataFrame to JSON with proper handling of special types
                data_dict = data.to_dict('records')
                with open(json_file, 'w') as f:
                    json.dump({
                        'pipeline_id': self.pipeline_id,
                        'processed_at': datetime.now().isoformat(),
                        'record_count': len(data_dict),
                        'orders': data_dict
                    }, f, indent=2, default=str)
                storage_summary['storage_operations'].append(f'Saved JSON: {json_file.name}')
                self.logger.info(f"💾 Saved JSON file: {json_file}")
        
        # 2. Database Storage (Simulated)
        if self.config.outputs['database']['enabled']:
            # In a real implementation, this would use the DatabaseManager
            # For demo purposes, we'll simulate database storage
            db_path = self.config.outputs['database']['path']
            table_name = self.config.outputs['database']['table']
            
            # Simulate database save
            storage_summary['storage_operations'].append(f'Saved to database: {table_name} ({len(data)} records)')
            self.logger.info(f"💾 Simulated database save: {len(data)} records to {table_name}")
        
        storage_summary['stored_records'] = len(data)
        storage_summary['success'] = True
        
    except Exception as e:
        error_msg = f"Storage failed: {str(e)}"
        storage_summary['errors'].append(error_msg)
        storage_summary['success'] = False
        self.logger.error(f"❌ {error_msg}")
    
    stage_time = time.time() - stage_start
    storage_summary['processing_time'] = stage_time
    
    self.results['storage'] = storage_summary
    
    self.logger.info(f"✅ Storage stage completed in {stage_time:.2f}s: {len(storage_summary['storage_operations'])} operations")

# Add method to pipeline builder class
DataPipelineBuilder.run_storage_stage = run_storage_stage

print("💾 Storage stage methods added to pipeline builder")

## 🚀 Step 6: Execute Complete Pipeline

Now let's put it all together and execute our complete data pipeline!

In [ ]:
# Add main pipeline execution method
def execute_pipeline(self):
    """Execute the complete data pipeline"""
    self.start_time = datetime.now()
    self.logger.info(f"🚀 Starting pipeline execution: {self.pipeline_id}")
    
    pipeline_summary = {
        'pipeline_id': self.pipeline_id,
        'start_time': self.start_time.isoformat(),
        'stages_completed': [],
        'stages_failed': [],
        'total_execution_time': 0,
        'final_record_count': 0,
        'overall_success': False
    }
    
    try:
        # Stage 1: Data Ingestion
        data = self.run_ingestion_stage()
        if self.results['ingestion']['success']:
            pipeline_summary['stages_completed'].append('ingestion')
        else:
            pipeline_summary['stages_failed'].append('ingestion')
            if len(data) == 0:
                raise Exception("No data ingested, stopping pipeline")
        
        # Stage 2: Data Validation
        data = self.run_validation_stage(data)
        if self.results['validation']['success']:
            pipeline_summary['stages_completed'].append('validation')
        else:
            pipeline_summary['stages_failed'].append('validation')
            # Continue with warnings but don't stop pipeline
            self.logger.warning("⚠️ Validation stage failed but continuing pipeline")
        
        # Stage 3: Data Transformation
        data = self.run_transformation_stage(data)
        if self.results['transformation']['success']:
            pipeline_summary['stages_completed'].append('transformation')
        else:
            pipeline_summary['stages_failed'].append('transformation')
            raise Exception("Transformation stage failed, stopping pipeline")
        
        # Stage 4: Data Storage
        self.run_storage_stage(data)
        if self.results['storage']['success']:
            pipeline_summary['stages_completed'].append('storage')
        else:
            pipeline_summary['stages_failed'].append('storage')
            raise Exception("Storage stage failed, stopping pipeline")
        
        # Pipeline completed successfully
        pipeline_summary['overall_success'] = True
        pipeline_summary['final_record_count'] = len(data)
        
    except Exception as e:
        pipeline_summary['overall_success'] = False
        pipeline_summary['error_message'] = str(e)
        self.logger.error(f"❌ Pipeline execution failed: {str(e)}")
    
    finally:
        self.end_time = datetime.now()
        pipeline_summary['end_time'] = self.end_time.isoformat()
        pipeline_summary['total_execution_time'] = (self.end_time - self.start_time).total_seconds()
        
        self.results['pipeline_summary'] = pipeline_summary
        
        # Log final results
        status = "✅ SUCCESS" if pipeline_summary['overall_success'] else "❌ FAILED"
        self.logger.info(f"{status} Pipeline execution completed in {pipeline_summary['total_execution_time']:.2f}s")
        
        return pipeline_summary, data if 'data' in locals() else pd.DataFrame()

# Add method to pipeline builder class
DataPipelineBuilder.execute_pipeline = execute_pipeline

# Execute the complete pipeline
print("🚀 Executing complete data pipeline...")
print("=" * 50)

pipeline_summary, final_data = pipeline_builder.execute_pipeline()

print("\n📊 PIPELINE EXECUTION RESULTS")
print("=" * 50)
print(f"Pipeline ID: {pipeline_summary['pipeline_id']}")
print(f"Status: {'✅ SUCCESS' if pipeline_summary['overall_success'] else '❌ FAILED'}")
print(f"Execution Time: {pipeline_summary['total_execution_time']:.2f} seconds")
print(f"Final Record Count: {pipeline_summary['final_record_count']}")
print(f"Stages Completed: {', '.join(pipeline_summary['stages_completed'])}")

if pipeline_summary['stages_failed']:
    print(f"Stages Failed: {', '.join(pipeline_summary['stages_failed'])}")

if 'error_message' in pipeline_summary:
    print(f"Error: {pipeline_summary['error_message']}")

## 📊 Step 7: Pipeline Results Analysis

Let's analyze the results of our pipeline execution and create visualizations to understand the data flow.

In [ ]:
# Analyze pipeline results
def analyze_pipeline_results(pipeline_builder, final_data):
    """Analyze and visualize pipeline results"""
    
    results = pipeline_builder.results
    
    print("📈 DETAILED STAGE ANALYSIS")
    print("=" * 60)
    
    # Ingestion Analysis
    if 'ingestion' in results:
        ing = results['ingestion']
        print(f"\n📥 INGESTION STAGE:")
        print(f"  Sources Processed: {ing.get('sources_processed', 0)}")
        print(f"  Successful Sources: {ing.get('successful_sources', 0)}")
        print(f"  Failed Sources: {ing.get('failed_sources', 0)}")
        print(f"  Total Records: {ing.get('total_records', 0)}")
        print(f"  Processing Time: {ing.get('processing_time', 0):.2f}s")
        if ing.get('errors'):
            print(f"  Errors: {len(ing['errors'])}")
    
    # Validation Analysis
    if 'validation' in results:
        val = results['validation']
        print(f"\n🔍 VALIDATION STAGE:")
        print(f"  Total Records: {val.get('total_records', 0)}")
        print(f"  Valid Records: {val.get('valid_records', 0)}")
        print(f"  Invalid Records: {val.get('invalid_records', 0)}")
        print(f"  Quality Score: {val.get('quality_score', 0):.1f}%")
        print(f"  Meets Threshold: {'✅ Yes' if val.get('meets_threshold', False) else '❌ No'}")
        print(f"  Processing Time: {val.get('processing_time', 0):.2f}s")
        if val.get('errors'):
            print(f"  Errors: {len(val['errors'])}")
    
    # Transformation Analysis
    if 'transformation' in results:
        trans = results['transformation']
        print(f"\n🧹 TRANSFORMATION STAGE:")
        print(f"  Original Records: {trans.get('original_records', 0)}")
        print(f"  Transformed Records: {trans.get('transformed_records', 0)}")
        print(f"  Operations Performed: {len(trans.get('operations_performed', []))}")
        print(f"  Processing Time: {trans.get('processing_time', 0):.2f}s")
        if trans.get('operations_performed'):
            print("  Operations:")
            for op in trans['operations_performed']:
                print(f"    - {op}")
    
    # Storage Analysis
    if 'storage' in results:
        stor = results['storage']
        print(f"\n💾 STORAGE STAGE:")
        print(f"  Total Records: {stor.get('total_records', 0)}")
        print(f"  Stored Records: {stor.get('stored_records', 0)}")
        print(f"  Storage Operations: {len(stor.get('storage_operations', []))}")
        print(f"  Processing Time: {stor.get('processing_time', 0):.2f}s")
        if stor.get('storage_operations'):
            print("  Operations:")
            for op in stor['storage_operations']:
                print(f"    - {op}")
    
    # Data Analysis
    if not final_data.empty:
        print(f"\n📊 FINAL DATA ANALYSIS:")
        print(f"  Total Records: {len(final_data)}")
        print(f"  Total Columns: {len(final_data.columns)}")
        
        # Show column info
        print(f"  Columns: {', '.join(final_data.columns.tolist())}")
        
        # Data quality analysis
        if 'quality_score' in final_data.columns:
            avg_quality = final_data['quality_score'].mean()
            min_quality = final_data['quality_score'].min()
            max_quality = final_data['quality_score'].max()
            print(f"  Average Quality Score: {avg_quality:.1f}")
            print(f"  Quality Range: {min_quality:.1f} - {max_quality:.1f}")
        
        # Business insights
        if 'total_amount' in final_data.columns:
            total_revenue = final_data['total_amount'].sum()
            avg_order_value = final_data['total_amount'].mean()
            print(f"  Total Revenue: ${total_revenue:,.2f}")
            print(f"  Average Order Value: ${avg_order_value:.2f}")
        
        if 'customer_segment' in final_data.columns:
            segment_dist = final_data['customer_segment'].value_counts()
            print(f"  Customer Segments: {dict(segment_dist)}")

# Run analysis
analyze_pipeline_results(pipeline_builder, final_data)

## 📈 Step 8: Pipeline Visualization

Let's create visualizations to better understand our pipeline performance and data insights.

In [ ]:
# Create pipeline visualizations
def create_pipeline_visualizations(pipeline_builder, final_data):
    """Create comprehensive pipeline visualizations"""
    
    results = pipeline_builder.results
    
    # Create figure with subplots
    fig = plt.figure(figsize=(20, 15))
    
    # 1. Pipeline Stage Performance
    ax1 = plt.subplot(2, 3, 1)
    stages = ['Ingestion', 'Validation', 'Transformation', 'Storage']
    times = [
        results.get('ingestion', {}).get('processing_time', 0),
        results.get('validation', {}).get('processing_time', 0),
        results.get('transformation', {}).get('processing_time', 0),
        results.get('storage', {}).get('processing_time', 0)
    ]
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    bars = ax1.bar(stages, times, color=colors, alpha=0.8)
    ax1.set_title('Pipeline Stage Performance', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Processing Time (seconds)')
    ax1.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, time in zip(bars, times):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{time:.2f}s', ha='center', va='bottom')
    
    # 2. Data Flow Through Pipeline
    ax2 = plt.subplot(2, 3, 2)
    flow_stages = ['Ingested', 'Validated', 'Transformed', 'Stored']
    record_counts = [
        results.get('ingestion', {}).get('total_records', 0),
        results.get('validation', {}).get('valid_records', 0),
        results.get('transformation', {}).get('transformed_records', 0),
        results.get('storage', {}).get('stored_records', 0)
    ]
    
    ax2.plot(flow_stages, record_counts, marker='o', linewidth=3, markersize=8, color='#FF6B6B')
    ax2.fill_between(flow_stages, record_counts, alpha=0.3, color='#FF6B6B')
    ax2.set_title('Data Flow Through Pipeline', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Number of Records')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    # Add value labels
    for i, count in enumerate(record_counts):
        ax2.annotate(f'{count}', (i, count), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontweight='bold')
    
    # 3. Data Quality Distribution
    if not final_data.empty and 'quality_score' in final_data.columns:
        ax3 = plt.subplot(2, 3, 3)
        quality_scores = final_data['quality_score']
        ax3.hist(quality_scores, bins=10, color='#4ECDC4', alpha=0.7, edgecolor='black')
        ax3.axvline(quality_scores.mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {quality_scores.mean():.1f}')
        ax3.set_title('Data Quality Score Distribution', fontsize=14, fontweight='bold')
        ax3.set_xlabel('Quality Score')
        ax3.set_ylabel('Number of Records')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    
    # 4. Customer Segment Analysis
    if not final_data.empty and 'customer_segment' in final_data.columns:
        ax4 = plt.subplot(2, 3, 4)
        segment_counts = final_data['customer_segment'].value_counts()
        colors_pie = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
        wedges, texts, autotexts = ax4.pie(segment_counts.values, labels=segment_counts.index, 
                                          autopct='%1.1f%%', colors=colors_pie, startangle=90)
        ax4.set_title('Customer Segment Distribution', fontsize=14, fontweight='bold')
        
        # Make percentage text bold
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
    
    # 5. Order Size Analysis
    if not final_data.empty and 'order_size' in final_data.columns:
        ax5 = plt.subplot(2, 3, 5)
        order_size_counts = final_data['order_size'].value_counts()
        bars = ax5.bar(order_size_counts.index, order_size_counts.values, 
                      color='#45B7D1', alpha=0.8)
        ax5.set_title('Order Size Distribution', fontsize=14, fontweight='bold')
        ax5.set_xlabel('Order Size Category')
        ax5.set_ylabel('Number of Orders')
        ax5.tick_params(axis='x', rotation=45)
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax5.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{int(height)}', ha='center', va='bottom', fontweight='bold')
    
    # 6. Revenue Analysis
    if not final_data.empty and 'total_amount' in final_data.columns:
        ax6 = plt.subplot(2, 3, 6)
        
        # Revenue by source if available
        if 'source' in final_data.columns:
            revenue_by_source = final_data.groupby('source')['total_amount'].sum()
            bars = ax6.bar(revenue_by_source.index, revenue_by_source.values, 
                          color='#96CEB4', alpha=0.8)
            ax6.set_title('Revenue by Source', fontsize=14, fontweight='bold')
            ax6.set_xlabel('Data Source')
            ax6.set_ylabel('Total Revenue ($)')
            ax6.tick_params(axis='x', rotation=45)
            
            # Add value labels
            for bar in bars:
                height = bar.get_height()
                ax6.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                        f'${height:,.0f}', ha='center', va='bottom', fontweight='bold')
        else:
            # Simple revenue histogram
            ax6.hist(final_data['total_amount'], bins=10, color='#96CEB4', alpha=0.7, edgecolor='black')
            ax6.set_title('Order Value Distribution', fontsize=14, fontweight='bold')
            ax6.set_xlabel('Order Value ($)')
            ax6.set_ylabel('Number of Orders')
            ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\n📊 PIPELINE PERFORMANCE SUMMARY")
    print("=" * 50)
    total_time = sum(times)
    print(f"Total Pipeline Time: {total_time:.2f} seconds")
    print(f"Average Stage Time: {total_time/len(times):.2f} seconds")
    print(f"Throughput: {record_counts[-1]/total_time:.1f} records/second")
    
    if not final_data.empty:
        print(f"\n💰 BUSINESS INSIGHTS")
        print("=" * 30)
        if 'total_amount' in final_data.columns:
            total_revenue = final_data['total_amount'].sum()
            avg_order = final_data['total_amount'].mean()
            print(f"Total Revenue Processed: ${total_revenue:,.2f}")
            print(f"Average Order Value: ${avg_order:.2f}")
            print(f"Revenue per Second: ${total_revenue/total_time:.2f}")

# Create visualizations
if not final_data.empty:
    create_pipeline_visualizations(pipeline_builder, final_data)
    
    # Display sample of final data
    print("\n📋 SAMPLE OF PROCESSED DATA")
    print("=" * 50)
    display(final_data.head())
else:
    print("⚠️ No data available for visualization")

## 🔧 Step 9: Error Handling and Recovery

Let's demonstrate how to handle errors and implement recovery mechanisms in our pipeline.

In [ ]:
# Demonstrate error handling and recovery
def demonstrate_error_handling():
    """Demonstrate various error scenarios and recovery mechanisms"""
    
    print("🔧 DEMONSTRATING ERROR HANDLING & RECOVERY")
    print("=" * 60)
    
    # Scenario 1: Missing data files
    print("\n📁 Scenario 1: Missing Data Files")
    print("-" * 40)
    
    # Create a pipeline with non-existent data sources
    error_config = PipelineConfig()
    error_config.data_sources['csv_files']['path'] = '../data/input/nonexistent/'
    
    error_pipeline = DataPipelineBuilder(error_config)
    
    try:
        data = error_pipeline.run_ingestion_stage()
        print(f"Result: {len(data)} records ingested")
        print(f"Errors: {len(error_pipeline.results['ingestion']['errors'])}")
        for error in error_pipeline.results['ingestion']['errors'][:3]:  # Show first 3 errors
            print(f"  - {error}")
    except Exception as e:
        print(f"Exception caught: {e}")
    
    # Scenario 2: Invalid data format
    print("\n📊 Scenario 2: Invalid Data Format")
    print("-" * 40)
    
    # Create invalid data
    invalid_data = pd.DataFrame({
        'order_id': ['', 'ORD-002', None],  # Missing IDs
        'customer_name': ['John Doe', '', 'Bob Wilson'],  # Missing name
        'product': ['iPhone 15', 'MacBook Pro', ''],  # Missing product
        'quantity': [-1, 'invalid', 2],  # Invalid quantity
        'price': [999.99, -100, 'free']  # Invalid price
    })
    
    validation_pipeline = DataPipelineBuilder(pipeline_config)
    validated_data = validation_pipeline.run_validation_stage(invalid_data)
    
    val_results = validation_pipeline.results['validation']
    print(f"Quality Score: {val_results['quality_score']:.1f}%")
    print(f"Valid Records: {val_results['valid_records']}/{val_results['total_records']}")
    print(f"Errors Found: {len(val_results['errors'])}")
    
    # Show some errors
    for error in val_results['errors'][:5]:  # Show first 5 errors
        print(f"  - {error}")
    
    # Scenario 3: Recovery strategies
    print("\n🔄 Scenario 3: Recovery Strategies")
    print("-" * 40)
    
    recovery_strategies = {
        'Data Quality Issues': [
            'Skip invalid records and continue processing',
            'Apply default values for missing fields',
            'Flag records for manual review',
            'Send alerts to data stewards'
        ],
        'Source Unavailability': [
            'Retry with exponential backoff',
            'Switch to backup data sources',
            'Continue with available sources',
            'Queue failed sources for later retry'
        ],
        'Processing Failures': [
            'Checkpoint and resume from last successful stage',
            'Rollback to previous known good state',
            'Partial processing with error reporting',
            'Circuit breaker to prevent cascade failures'
        ],
        'Storage Issues': [
            'Write to alternative storage locations',
            'Buffer data in memory/temporary files',
            'Compress data to reduce storage requirements',
            'Implement storage quotas and cleanup'
        ]
    }
    
    for scenario, strategies in recovery_strategies.items():
        print(f"\n🎯 {scenario}:")
        for i, strategy in enumerate(strategies, 1):
            print(f"  {i}. {strategy}")
    
    # Scenario 4: Circuit breaker pattern
    print("\n⚡ Scenario 4: Circuit Breaker Implementation")
    print("-" * 50)
    
    class CircuitBreaker:
        def __init__(self, failure_threshold=3, recovery_timeout=60):
            self.failure_threshold = failure_threshold
            self.recovery_timeout = recovery_timeout
            self.failure_count = 0
            self.last_failure_time = None
            self.state = 'CLOSED'  # CLOSED, OPEN, HALF_OPEN
        
        def call(self, func, *args, **kwargs):
            if self.state == 'OPEN':
                if time.time() - self.last_failure_time > self.recovery_timeout:
                    self.state = 'HALF_OPEN'
                else:
                    raise Exception("Circuit breaker is OPEN")
            
            try:
                result = func(*args, **kwargs)
                self.on_success()
                return result
            except Exception as e:
                self.on_failure()
                raise e
        
        def on_success(self):
            self.failure_count = 0
            self.state = 'CLOSED'
        
        def on_failure(self):
            self.failure_count += 1
            self.last_failure_time = time.time()
            if self.failure_count >= self.failure_threshold:
                self.state = 'OPEN'
    
    # Demonstrate circuit breaker
    circuit_breaker = CircuitBreaker(failure_threshold=2, recovery_timeout=5)
    
    def unreliable_function():
        import random
        if random.random() < 0.7:  # 70% failure rate
            raise Exception("Service unavailable")
        return "Success!"
    
    print("Testing circuit breaker with unreliable function:")
    for i in range(5):
        try:
            result = circuit_breaker.call(unreliable_function)
            print(f"  Attempt {i+1}: {result} (State: {circuit_breaker.state})")
        except Exception as e:
            print(f"  Attempt {i+1}: Failed - {e} (State: {circuit_breaker.state})")
        time.sleep(0.1)  # Small delay

# Run error handling demonstration
demonstrate_error_handling()

## 📋 Step 10: Pipeline Report Generation

Let's create a comprehensive report of our pipeline execution that can be saved and shared.

In [ ]:
# Generate comprehensive pipeline report
import os
from pathlib import Path
from datetime import datetime

def generate_pipeline_report(pipeline_builder, final_data):
    """Generate a comprehensive pipeline execution report"""

    results = pipeline_builder.results
    pipeline_summary = results.get('pipeline_summary', {})

    report = []
    report.append("# 📊 Data Pipeline Execution Report")
    report.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report.append("")

    # Executive Summary
    report.append("## 🎯 Executive Summary")
    report.append(f"- **Pipeline ID**: {pipeline_builder.pipeline_id}")
    report.append(f"- **Status**: {'✅ SUCCESS' if pipeline_summary.get('overall_success', False) else '❌ FAILED'}")
    report.append(f"- **Execution Time**: {pipeline_summary.get('total_execution_time', 0):.2f} seconds")
    report.append(f"- **Records Processed**: {pipeline_summary.get('final_record_count', 0):,}")
    report.append(f"- **Stages Completed**: {len(pipeline_summary.get('stages_completed', []))}")
    report.append(f"- **Stages Failed**: {len(pipeline_summary.get('stages_failed', []))}")
    report.append("")

    # Stage Details
    report.append("## 🔄 Stage Execution Details")

    # Ingestion Stage
    if 'ingestion' in results:
        ing = results['ingestion']
        report.append("### 📥 Data Ingestion Stage")
        report.append(f"- **Status**: {'✅ Success' if ing.get('success', False) else '❌ Failed'}")
        report.append(f"- **Sources Processed**: {ing.get('sources_processed', 0)}")
        report.append(f"- **Successful Sources**: {ing.get('successful_sources', 0)}")
        report.append(f"- **Failed Sources**: {ing.get('failed_sources', 0)}")
        report.append(f"- **Total Records**: {ing.get('total_records', 0):,}")
        report.append(f"- **Processing Time**: {ing.get('processing_time', 0):.2f} seconds")
        if ing.get('errors'):
            report.append(f"- **Errors**: {len(ing['errors'])}")
        report.append("")

    # Validation Stage
    if 'validation' in results:
        val = results['validation']
        report.append("### 🔍 Data Validation Stage")
        report.append(f"- **Status**: {'✅ Success' if val.get('success', False) else '❌ Failed'}")
        report.append(f"- **Total Records**: {val.get('total_records', 0):,}")
        report.append(f"- **Valid Records**: {val.get('valid_records', 0):,}")
        report.append(f"- **Invalid Records**: {val.get('invalid_records', 0):,}")
        report.append(f"- **Quality Score**: {val.get('quality_score', 0):.1f}%")
        report.append(f"- **Meets Threshold**: {'✅ Yes' if val.get('meets_threshold', False) else '❌ No'}")
        report.append(f"- **Processing Time**: {val.get('processing_time', 0):.2f} seconds")
        if val.get('errors'):
            report.append(f"- **Validation Errors**: {len(val['errors'])}")
        report.append("")

    # Transformation Stage
    if 'transformation' in results:
        trans = results['transformation']
        report.append("### 🧹 Data Transformation Stage")
        report.append(f"- **Status**: {'✅ Success' if trans.get('success', False) else '❌ Failed'}")
        report.append(f"- **Original Records**: {trans.get('original_records', 0):,}")
        report.append(f"- **Transformed Records**: {trans.get('transformed_records', 0):,}")
        report.append(f"- **Operations Performed**: {len(trans.get('operations_performed', []))}")
        report.append(f"- **Processing Time**: {trans.get('processing_time', 0):.2f} seconds")

        if trans.get('operations_performed'):
            report.append("- **Operations**:")
            for op in trans['operations_performed']:
                report.append(f"  - {op}")
        report.append("")

    # Storage Stage
    if 'storage' in results:
        stor = results['storage']
        report.append("### 💾 Data Storage Stage")
        report.append(f"- **Status**: {'✅ Success' if stor.get('success', False) else '❌ Failed'}")
        report.append(f"- **Records Stored**: {stor.get('stored_records', 0):,}")
        report.append(f"- **Storage Operations**: {len(stor.get('storage_operations', []))}")
        report.append(f"- **Processing Time**: {stor.get('processing_time', 0):.2f} seconds")

        if stor.get('storage_operations'):
            report.append("- **Operations**:")
            for op in stor['storage_operations']:
                report.append(f"  - {op}")
        report.append("")

    # Data Quality Analysis
    if not final_data.empty:
        report.append("## 📊 Data Quality Analysis")
        report.append(f"- **Total Records**: {len(final_data):,}")
        report.append(f"- **Total Columns**: {len(final_data.columns)}")

        if 'quality_score' in final_data.columns:
            avg_quality = final_data['quality_score'].mean()
            min_quality = final_data['quality_score'].min()
            max_quality = final_data['quality_score'].max()
            report.append(f"- **Average Quality Score**: {avg_quality:.1f}/100")
            report.append(f"- **Quality Range**: {min_quality:.1f} - {max_quality:.1f}")

            # Quality distribution
            excellent = (final_data['quality_score'] >= 95).sum()
            good = ((final_data['quality_score'] >= 80) & (final_data['quality_score'] < 95)).sum()
            fair = ((final_data['quality_score'] >= 60) & (final_data['quality_score'] < 80)).sum()
            poor = (final_data['quality_score'] < 60).sum()

            report.append(f"- **Quality Distribution**:")
            report.append(f"  - Excellent (95-100): {excellent} records ({excellent/len(final_data)*100:.1f}%)")
            report.append(f"  - Good (80-94): {good} records ({good/len(final_data)*100:.1f}%)")
            report.append(f"  - Fair (60-79): {fair} records ({fair/len(final_data)*100:.1f}%)")
            report.append(f"  - Poor (<60): {poor} records ({poor/len(final_data)*100:.1f}%)")

        report.append("")

    # Business Insights
    if not final_data.empty:
        report.append("## 💰 Business Insights")

        if 'total_amount' in final_data.columns:
            total_revenue = final_data['total_amount'].sum()
            avg_order_value = final_data['total_amount'].mean()
            median_order_value = final_data['total_amount'].median()

            report.append(f"- **Total Revenue**: ${total_revenue:,.2f}")
            report.append(f"- **Average Order Value**: ${avg_order_value:.2f}")
            report.append(f"- **Median Order Value**: ${median_order_value:.2f}")

        if 'customer_segment' in final_data.columns:
            segment_dist = final_data['customer_segment'].value_counts()
            report.append(f"- **Customer Segments**:")
            for segment, count in segment_dist.items():
                percentage = (count / len(final_data)) * 100
                report.append(f"  - {segment}: {count} customers ({percentage:.1f}%)")

        if 'source' in final_data.columns:
            source_dist = final_data['source'].value_counts()
            report.append(f"- **Data Sources**:")
            for source, count in source_dist.items():
                percentage = (count / len(final_data)) * 100
                report.append(f"  - {source}: {count} records ({percentage:.1f}%)")

        report.append("")

    # Performance Metrics
    report.append("## ⚡ Performance Metrics")
    total_time = pipeline_summary.get('total_execution_time', 0)
    total_records = pipeline_summary.get('final_record_count', 0)

    if total_time > 0:
        throughput = total_records / total_time
        report.append(f"- **Throughput**: {throughput:.1f} records/second")
        report.append(f"- **Processing Rate**: {total_records/total_time*60:.0f} records/minute")

    # Stage performance breakdown
    stage_times = {
        'Ingestion': results.get('ingestion', {}).get('processing_time', 0),
        'Validation': results.get('validation', {}).get('processing_time', 0),
        'Transformation': results.get('transformation', {}).get('processing_time', 0),
        'Storage': results.get('storage', {}).get('processing_time', 0)
    }

    report.append(f"- **Stage Performance**:")
    for stage, time_taken in stage_times.items():
        if total_time > 0:
            percentage = (time_taken / total_time) * 100
            report.append(f"  - {stage}: {time_taken:.2f}s ({percentage:.1f}% of total)")

    report.append("")

    # Recommendations
    report.append("## 💡 Recommendations")

    if pipeline_summary.get('overall_success', False):
        report.append("- ✅ Pipeline executed successfully")

        # Performance recommendations
        if total_time > 0 and total_records > 0:
            throughput = total_records / total_time
            if throughput < 10:
                report.append("- ⚡ Consider optimizing processing speed (current throughput is low)")
            if stage_times['Validation'] > stage_times['Transformation']:
                report.append("- 🔍 Validation stage is taking longer than transformation - consider optimizing validation rules")

        # Data quality recommendations
        if not final_data.empty and 'quality_score' in final_data.columns:
            avg_quality = final_data['quality_score'].mean()
            if avg_quality < 80:
                report.append("- 📊 Data quality is below recommended threshold - review data sources")
            elif avg_quality >= 95:
                report.append("- 🌟 Excellent data quality maintained")

        report.append("- 📈 Monitor pipeline performance trends over time")
        report.append("- 🔄 Consider scheduling regular pipeline runs")

    else:
        report.append("- ❌ Pipeline execution failed")
        report.append("- 🔍 Review error logs and fix issues")
        report.append("- 🔄 Implement retry mechanisms for failed stages")
        report.append("- 📞 Contact data engineering team if issues persist")

    report.append("")
    report.append("---")
    report.append(f"*Report generated by Data Pipeline Builder v1.0*")

    return "\n".join(report)

# Generate and display report
try:
    pipeline_report = generate_pipeline_report(pipeline_builder, final_data)

    print("📋 COMPREHENSIVE PIPELINE REPORT")
    print("=" * 60)
    print(pipeline_report)

    # Save report to file with better path handling
    current_dir = Path.cwd()

    # If we're in notebooks folder, go up one level
    if current_dir.name == 'notebooks':
        base_dir = current_dir.parent
    else:
        base_dir = current_dir

    # Create output directory if it doesn't exist
    output_dir = base_dir / 'data' / 'output'
    output_dir.mkdir(parents=True, exist_ok=True)

    # Generate report filename
    report_filename = output_dir / f"pipeline_report_{pipeline_builder.pipeline_id}.md"

    # Save report
    with open(report_filename, 'w', encoding='utf-8') as f:
        f.write(pipeline_report)

    print(f"\n💾 Report saved to: {report_filename}")

except Exception as e:
    print(f"\n❌ Error generating pipeline report: {e}")
    print(f"Current working directory: {Path.cwd()}")

    # Try to display basic information if available
    try:
        if 'pipeline_builder' in locals() and hasattr(pipeline_builder, 'pipeline_id'):
            print(f"Pipeline ID: {pipeline_builder.pipeline_id}")
        if 'final_data' in locals() and not final_data.empty:
            print(f"Final data shape: {final_data.shape}")
            print(f"Final data columns: {list(final_data.columns)}")
    except:
        pass

## 🎯 Key Takeaways

Congratulations! You've successfully built and executed a complete data ingestion pipeline. Here's what you've accomplished:

### ✅ **Pipeline Components Mastered**
- **📥 Data Ingestion**: Multi-source data collection with error handling
- **🔍 Data Validation**: Quality scoring and business rule validation
- **🧹 Data Transformation**: Cleaning, standardization, and enrichment
- **💾 Data Storage**: Multiple output formats and destinations
- **📊 Monitoring**: Performance tracking and comprehensive reporting

### ✅ **Production-Ready Features**
- **Error Handling**: Graceful failure management and recovery
- **Performance Monitoring**: Stage-by-stage timing and throughput analysis
- **Data Quality**: Comprehensive quality scoring and reporting
- **Configurability**: Flexible configuration for different environments
- **Observability**: Detailed logging and execution reports

### ✅ **Business Value**
- **Automated Processing**: Hands-off data processing workflows
- **Quality Assurance**: Built-in data quality checks and scoring
- **Scalability**: Modular design supporting growth
- **Reliability**: Error recovery and fault tolerance
- **Insights**: Business intelligence and performance metrics

---

## 🚀 What's Next?

In the final tutorial, **"07_monitoring_pipeline.ipynb"**, you'll learn:
- 📈 **Advanced Monitoring**: Real-time pipeline health monitoring
- 🚨 **Alerting Systems**: Automated notifications for issues
- 📊 **Dashboard Creation**: Visual monitoring dashboards
- 🔍 **Performance Optimization**: Identifying and fixing bottlenecks
- 📋 **Operational Procedures**: Best practices for production operations

### 🎯 **Practice Exercises**

Before moving to the next tutorial, try these exercises:

1. **Extend the Pipeline**: Add a new data source (e.g., XML files)
2. **Custom Validation**: Create domain-specific validation rules
3. **Performance Testing**: Test with larger datasets
4. **Error Scenarios**: Simulate various failure conditions
5. **Configuration**: Create different configurations for dev/prod

---

## 📚 Additional Resources

- **🏗️ Architecture**: Study enterprise data pipeline patterns
- **⚡ Performance**: Learn about parallel processing and optimization
- **🔒 Security**: Implement data encryption and access controls
- **☁️ Cloud**: Deploy pipelines to cloud platforms (AWS, GCP, Azure)
- **🔄 Orchestration**: Explore Apache Airflow, Prefect, or Dagster

---

**Excellent work building your complete data pipeline! 🎉**